In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time

In [5]:
def step3_push_excel(Name, Dosage, Component, Way_Taking, Type_Form, Zheng_Form, Illness, row_index):
    """
    将爬取到的内容放入表格
    :param row_index:
    :param Name:
    :param Dosage:
    :param Component:
    :param Way_Taking:
    :param Type_Form:
    :param Zheng_Form:
    :param Illness:
    :return:
    """
    data_ETCM = pd.read_excel("D:/github_planting/MGM/data/data_ETCM.xlsx", sheet_name=0)

    data_ETCM.loc[row_index, '方剂名称'] = Name[0]
    data_ETCM.loc[row_index, '剂型'] = Dosage[0]
    data_ETCM.loc[row_index, '使用方法'] = Way_Taking[0]
    data_ETCM.loc[row_index, '方剂类型'] = Type_Form[0]
    data_ETCM.loc[row_index, '对证'] = Zheng_Form[0]
    data_ETCM.loc[row_index, '临床表现'] = Illness[0]

    for i in range(len(Component)):
        data_ETCM.loc[row_index, i+6] = Component[i]

    data_ETCM.to_excel("D:/github_planting/MGM/data/data_ETCM.xlsx", index=False)

In [6]:
def step2_get_infor(page_content, TCM_index):
    """
    获取各类方剂及其对应信息
    :param TCM_index:
    :param page_content: 网页源码
    :return:
    """
    # 获取方剂名称信息
    Name_Form_pattern = r"<a.*?href=.*?target='_blank'>(.*?)<\/a>"
    Name_Form_matches = re.findall(Name_Form_pattern, page_content)

    # 获取方剂剂型
    Dosage_Form_pattern = r'Dosage Form.*?>.*?>(.*?)<\/div>'
    Dosage_Form_matches = re.findall(Dosage_Form_pattern, page_content)

    # 获取方剂组成
    Component_Form_pattern = r"html\?id=.*?>(.*?)</a>"
    Component_Form_matches = re.findall(Component_Form_pattern, page_content)
    Component_Form_matches = Component_Form_matches[0:int(len(Component_Form_matches)/2)]

    # 获取服用方法
    Way_Taking_Form_pattern = r"Administration</div>.*?>(.*?)</div>"
    Way_Taking_Form_matches = re.findall(Way_Taking_Form_pattern, page_content)

    # 获取药物类型
    Type_Form_pattern = r"id='L778'.*?Type.*?>.*?>(.*?)</div>"
    Type_Form_matches = re.findall(Type_Form_pattern, page_content)

    # 获取药物证相
    Zheng_Form_pattern = r"id='L779'.*?Syndromes in Chinese.*?>.*?>(.*?)</div>"
    Zheng_Form_matches = re.findall(Zheng_Form_pattern, page_content)

    # 获取药物对症
    Illness_Form_pattern = r"id='L7711'.*?Indications in Chinese.*?>.*?>(.*?)</div>"
    Illness_Form_matches = re.findall(Illness_Form_pattern, page_content)

    step3_push_excel(Name_Form_matches, Dosage_Form_matches, Component_Form_matches, Way_Taking_Form_matches, Type_Form_matches, Zheng_Form_matches,Illness_Form_matches, TCM_index)

    return 0

    #return Name_Form_matches, Dosage_Form_matches, Component_Form_matches, Way_Taking_Form_matches, Type_Form_matches, Zheng_Form_matches,Illness_Form_matches

with open("D:/github_planting/MGM/data/测试.txt", "r", encoding='utf-8')as f:
    content = f.read()
step2_get_infor(content, 1)

0

In [5]:
def step1_visit_website(target_url, TCM_index):
    """
    访问网站链接并获取网页源码
    :param TCM_index:
    :param target_url:网址链接
    :return: 网页源代码的txt格式
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}

    proxies = {"http": None, "https": None}
    res = requests.get(url, headers=headers,proxies=proxies)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.content, 'html.parser')
    script_tags = soup.find_all('script')[10]

    script_tags = str(script_tags)[:30000]

    step2_get_infor(script_tags, TCM_index)

    return 0

    #return Name_Form_matches, Dosage_Form_matches, Component_Form_matches, Way_Taking_Form_matches, Type_Form_matches, Zheng_Form_matches, Illness_Form_matches

url = 'http://www.tcmip.cn/ETCM/index.php/Home/Index/fj_details.html?id=4'
step1_visit_website(url, 1)

NameError: name 'step2_get_infor' is not defined

# main函数

In [9]:
start_time = time.time()

base_url = 'http://www.tcmip.cn/ETCM/index.php/Home/Index/fj_details.html?id='

for i in range(3970):
    extra_url = str(i + 1)
    url = base_url + extra_url
    try:
        print("第", i+1, "个网站已成功爬取")
        url_content = step1_visit_website(url, i+1)
    except:
        print("第", i+1, "个网站爬取失败")
        url_content = step1_visit_website(url, i+1)

end_time = time.time()
print(str(end_time - start_time))

第 3963 个网站已成功爬取
第 3964 个网站已成功爬取
第 3965 个网站已成功爬取
第 3966 个网站已成功爬取
第 3967 个网站已成功爬取
第 3968 个网站已成功爬取
第 3969 个网站已成功爬取
第 3970 个网站已成功爬取
5.17333722114563
